# MIBI and IMC

In [ ]:
library(spatstat)
library(dplyr)
#library(ggplot2)
#library(imager,quietly = True)
library(stringr)
#ulimit::memory_limit(6000)

In [ ]:
mydata <- read.csv("./data/20220420_JP-TMAs_IMC-TMAs_MIBI_CombinedCelltypes_all.csv")


In [ ]:
#df[, 12:18][df[, 12:18] == 0] <- NA
#simplify to tumor
mydata[,'leiden'][mydata[,'leidencelltype5'] == 'epithelial'] <- 'tumor'

In [ ]:
# not cycIF
mydata <- mydata[!grepl("JP-TMA", mydata$X),]
head(mydata)
tail(mydata)

## start here

4/19/2023

In [ ]:
#### all the measures
ls_scene = unique(mydata$slide_scene)
ls_cells = unique(mydata$leiden) #c('T cell','B cell')#,'tumor','Endothelial','CD68+ immune','fibroblast','CD45 low immune'
ls_from = unique(mydata$leiden) #c('T cell','B cell','tumor')#
#ls_cells = c('tumor','Quies. str.','Vim+ FB','Macrophage','endothelial','CD3 T cell')#,'CD4 T cell','CD8 T cell''FN+ FB',
#ls_from = c('tumor','CD3 T cell') #,'CD4 T cell','CD8 T cell'
combined_df_Kcross <- data.frame(r=double(),
                    theo=double(),
                    iso=double(),
                    slide_scene=character(),
                    cell=character())
combined_df_Kest <- data.frame(r=double(),
                    theo=double(),
                    iso=double(),
                    slide_scene=character(),
                    cell=character())
combined_df_density <- data.frame(d=double(),
                    slide_scene=character(),
                    cell=character())

combined_df_Gcross <- data.frame(d=double(),
                    slide_scene=character(),
                    cell=character())

# loop version 1
datalist = list()
for (s_scene in ls_scene) { #ls_scene[1:18]
  print(s_scene)
  s_scene_scene = str_replace(s_scene, '_', '_scene')
  s_scene_scene = str_replace(s_scene_scene, '_scenescene', '_scene')
  print(s_scene_scene)
  mydataSubset <- mydata[grep(s_scene_scene, mydata$X), c('X','DAPI_X','DAPI_Y','leiden')]
  mymin <- mydataSubset %>% summarise_if(is.numeric, min)
  mymax <- mydataSubset %>% summarise_if(is.numeric, max)
  mypattern <- ppp(mydataSubset[,"DAPI_X"],mydataSubset[,"DAPI_Y"],c(mymin %>% pull('DAPI_X'),mymax %>% pull('DAPI_X')),
                c(mymin %>% pull('DAPI_Y'),mymax %>% pull('DAPI_Y')))
  win_p <- mypattern$window
  marks(mypattern) <- factor(mydataSubset$leiden)
  startTime <- Sys.time()
  for (s_cell in intersect((unique(mydataSubset$leiden)),ls_cells)) {
      print(s_cell)
      for (s_center in intersect((unique(mydataSubset$leiden)),ls_from)) {
          print(s_center)
          if (s_cell!=s_center){
              print(s_cell)
              print(s_center)
              # kcross
              Kcross_cell <- Kcross(X=mypattern, from=s_center, to=s_cell,correction='isotropic',rmax=100,nlarge=3000)
              Kcross_cell['slide_scene'] = s_scene
              Kcross_cell['from_cell'] = s_center
              Kcross_cell['to_cell'] = s_cell
              combined_df_Kcross <- rbind(combined_df_Kcross, Kcross_cell)
              #g cross
              Gcross_cell <- Gcross(X=mypattern, i=s_center, j=s_cell, correction=c("rs", "km", "han"))
              Gcross_cell['slide_scene'] = s_scene
              Gcross_cell['from_cell'] = s_center
              Gcross_cell['to_cell'] = s_cell
              combined_df_Gcross <- rbind(combined_df_Gcross, Gcross_cell)
              }
          else {
            mydata_cell = mydataSubset[grep(s_cell, mydataSubset$leiden), ]
            mypatternK <- ppp(x=(mydata_cell[,"DAPI_X"]),y=((mydata_cell[,"DAPI_Y"])),window=win_p)
            Kest_cell <- Kest(mypatternK,correction = 'isotropic',rmax=100,nlarge=3000)
            #fit <- ppm(mypatternK, ~ polynom(x,y,2), Poisson())
            #lambda <- predict(fit, locations=mypatternK, type="trend")
            #Kest_cell <- Kinhom(mypatternK,lambda,correction = 'isotropic',rmax=100/.325,nlarge=4000) #inhom
            Kest_cell['slide_scene'] = s_scene
            Kest_cell['cell'] = s_cell
            combined_df_Kest <- rbind(combined_df_Kest, Kest_cell)
            D <- intensity(mypatternK)
            combined_df_density[nrow(combined_df_density) + 1,] <- list(D, s_scene, s_cell)
             }
          }
      
  }
  endTime <- Sys.time()
  print(endTime - startTime)
  #break
    }


In [ ]:
# #save out
# s_num = 'ROI'#'inhom' #first
# write.csv(combined_df_Kcross,sprintf("./data/IMC-MIBI_%s_Kcross.csv",s_num), row.names = FALSE)
# write.csv(combined_df_Kest,sprintf("./data/IMC-MIBI_%s_Kest.csv",s_num), row.names = FALSE)
# write.csv(combined_df_density,sprintf("./data/IMC-MIBI_%s_density.csv",s_num), row.names = FALSE)
# write.csv(combined_df_Gcross,sprintf("./data/IMC-MIBI_%s_Gcross.csv",s_num), row.names = FALSE)

In [ ]:
print('done')

## Occupancy

In [ ]:
makeQuadrats <- function(p, squareLength) {
  require(spatstat)
  
  dx <- Window(p)$xrange[2] - Window(p)$xrange[1]
  dy <- Window(p)$yrange[2] - Window(p)$yrange[1]
  
  nx <- floor(dx/squareLength)
  ny <- floor(dy/squareLength)
  return(c(nx,ny))
 }

In [ ]:
#### occupancy counts
ls_scene = unique(mydata$slide_scene)
mydata[,'leiden2'] = sub("$","_",mydata$leiden)
combined_df_occ <- data.frame(unlist.result_table.=double(),
                    slide_scene=character())
# loop 
pList = list()
for (s_scene in ls_scene) { #ls_scene[1:18]
  print(s_scene)
  s_scene_scene = str_replace(s_scene, '_', '_scene')
  s_scene_scene = str_replace(s_scene_scene, '_scenescene', '_scene')
  print(s_scene_scene)
  mydataSubset <- mydata[grep(s_scene_scene, mydata$X), c('X','DAPI_X','DAPI_Y','leiden2')]
  mymin <- mydataSubset %>% summarise_if(is.numeric, min)
  mymax <- mydataSubset %>% summarise_if(is.numeric, max)
  mypattern <- ppp(mydataSubset[,"DAPI_X"],mydataSubset[,"DAPI_Y"],c(mymin %>% pull('DAPI_X'),mymax %>% pull('DAPI_X')),
                c(mymin %>% pull('DAPI_Y'),mymax %>% pull('DAPI_Y')))
  win_p <- mypattern$window
  marks(mypattern) <- factor(mydataSubset$leiden2)
  nxny <- makeQuadrats(mypattern,50)
  result_table <- quadratcount.splitppp(split(mypattern), nx=nxny[1], ny=nxny[2])
  counts <- data.frame(unlist(result_table))
  if (length(counts) > 0) {
    counts['slide_scene'] = s_scene
    combined_df_occ <- rbind(combined_df_occ, counts)
    }
  
  #break
  }

# #save out
s_num = 'ROI'#'inhom' #first
write.csv(combined_df_occ,sprintf("./data/IMC-MIBI_%s_Occ.csv",s_num), row.names = TRUE)
print('done')